In [1]:
!pip install uproot tqdm numexpr

In [2]:
import uproot
import pandas as pd
import tqdm
from pathlib import Path
import numpy as np
import numexpr

In [3]:
relations_file = uproot.open("/home/jonas/Documents/physics/lhcb/dfei/irishep/data/IRIS-HEP_DFEI/relations_data.root")

In [4]:
relations_tree = relations_file['ParticleRelations']

In [5]:
relevant_relations_keys = [
                          'DOCA_reco',
                          'ExpandedEventNumber',
                          'FirstParticleIndex',
                          'FromSameAssociatedPV_reco',
                          'SecondParticleIndex',
                          'delta_z0_reco',
                          'p1_isCharged',
                          'p2_isCharged',
                          'theta_reco',
                          'trdist_reco'
                          ]

In [6]:
df_relations = relations_tree.arrays(relevant_relations_keys, library='pd')

In [7]:
num_of_events = df_relations['ExpandedEventNumber'].max()

In [8]:
num_of_events = int(num_of_events)

In [9]:
K = 10

In [10]:
df_relations.sort_values(by=["ExpandedEventNumber"], inplace=True, ascending=True)

In [11]:
import time
lower_index = 0
old_event_number = df_relations.ExpandedEventNumber.iloc[0]
sorter = np.arange(0, df_relations.shape[0], step=1)
event_nrs = df_relations.ExpandedEventNumber
sorted_indices = np.searchsorted(event_nrs, sorter, side="right", sorter=sorter)
for event_number in tqdm.tqdm(range(num_of_events)):
  start = time.time()
  
  upper_index = sorted_indices[event_number]
  df_r = df_relations.iloc[lower_index:upper_index]
  lower_index = upper_index
  old_event_number = df_relations.iloc[upper_index]
  unique_particle_indexes = pd.concat([df_r['FirstParticleIndex'], df_r['SecondParticleIndex']]).unique()

  indices = np.empty(shape=(K * unique_particle_indexes.shape[0],), dtype=np.int64)
  first_p_index = df_r.FirstParticleIndex
  second_p_index = df_r.SecondParticleIndex

  lower_idx = 0  
  for i, p_idx in enumerate(unique_particle_indexes):

    criterion = numexpr.evaluate("(first_p_index == p_idx) | (second_p_index == p_idx)")
    k = K if K < np.count_nonzero(criterion) else np.count_nonzero(criterion)
    upper_idx = lower_idx + k
    indices[lower_idx: upper_idx] = df_r.loc[criterion].nsmallest(k, "DOCA_reco").index
    lower_idx = upper_idx
  indices = np.unique(indices)
  filtered_event_relations_df = df_r.loc[indices]
  filename = f'event_{event_number}_top_{K}_relations_filtered_by_DOCA_for_each_particle.csv'
  path = f'/home/jonas/Documents/physics/lhcb/dfei/irishep/data/IRIS-HEP_DFEI/filtered_relations_by_DOCA/{filename}'
  filtered_event_relations_df.to_csv(path)

100%|███████████████████████████████████████████████████████████████████████████████| 6161/6161 [40:26<00:00,  2.54it/s]
